# Test Web Research Agent

This notebook tests the web research agent created in the previous step by asking it to research the latest 3M news and displaying the response in markdown format.

**Prerequisites:**
- Web research agent must be created (from 01_agent_creation)
- Agent ID must be saved in .env file as WEB_RESEARCH_AGENT_ID

## Import required packages

In [46]:
import os
import time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables
load_dotenv('../../.env', override=True)

True

## Create Azure AI Project client

In [47]:
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

## Load the Web Research Agent

In [48]:
# Get the agent ID from environment variables
agent_id = os.environ["WEB_RESEARCH_AGENT_ID"]
print(f"Using Web Research Agent ID: {agent_id}")

# Verify the agent exists
agent = project_client.agents.get_agent(agent_id)
print(f"Agent Name: {agent.name}")
print(f"Agent Model: {agent.model}")

Using Web Research Agent ID: asst_IRw9iGlx3q1Dqdoyoutgu4so
Agent Name: web-research-agent
Agent Model: gpt-4o


## Create a thread and test the agent

In [49]:
# Create a new thread for this test
thread = project_client.agents.threads.create()
print(f"Created thread: {thread.id}")

Created thread: thread_fLsRpBJyOYSZkMKeW6YCjZAs


In [50]:
# Ask the default question about 3M news
test_question = "What is the latest in 3M news?"
print(f"Asking question: {test_question}")

# Create the message
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=test_question
)

print(f"Message created: {message.id}")

Asking question: What is the latest in 3M news?
Message created: msg_yoJA6aCN8zLQ7G6gJjRZL9S0


In [51]:
# Run the agent
run = project_client.agents.runs.create(
    thread_id=thread.id,
    agent_id=agent_id
)

print(f"Run ID: {run.id}")
print("Research in progress...")

Run ID: run_Hz7OOvS1MIeO3otQ83R13F3I
Research in progress...


In [52]:
# Wait for completion
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(3)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Status: {run.status}")

print(f"\nFinal status: {run.status}")

Status: RunStatus.IN_PROGRESS
Status: RunStatus.COMPLETED

Final status: RunStatus.COMPLETED


## Display the research results

In [60]:
# Get the messages from the thread
messages = project_client.agents.messages.list(thread_id=thread.id)

# Find the assistant's response
for message in messages:
    if message.role != "assistant":
        continue
    citations = message.url_citation_annotations
    if len(citations) > 0:
        citations = [x.as_dict() for x in citations]
    updated_text = ''
    if len(message.content) > 0:
        for entity in message.content:
            if entity.type=='text':
                updated_text += entity.text.value
    for citation in citations:
        formatted_citation = f" [{citation['url_citation']['title']}]({citation['url_citation']['url']}) "
        updated_text = updated_text.replace(citation['text'], formatted_citation)
    research_response = updated_text
    

print("Research completed! Displaying results below...")

Research completed! Displaying results below...


## 3M News Research Results

In [61]:
# Display the research results as formatted markdown
display(Markdown(research_response))

Recent developments in 3M include:

1. **3M Young Scientist Challenge**: On June 25, 2025, 3M and Discovery Education announced the top 10 finalists for the 2025 3M Young Scientist Challenge, a premier middle school science competition [3M News Center - Press Releases](https://news.3m.com/press-releases) .

2. **Structural Reorganization**: 3M has been undergoing structural changes to streamline its operations, including reducing its corporate center size, optimizing manufacturing roles, and simplifying its supply chain. These efforts have improved margins and cash flow, with a projected adjusted earnings range of $7.60-$7.90 per share for 2025 [3M Structural Reorganization Actions in Motion: Will It Boost ... - Nasdaq](https://www.nasdaq.com/articles/3m-structural-reorganization-actions-motion-will-it-boost-margins) .

3. **Tariff Challenges**: The company expects up to $850 million in tariff-related expenses in 2025, primarily due to U.S.-China trade tensions. 3M is leveraging its global manufacturing and distribution network to mitigate these impacts [3M expects up to $850M tariff hit | Manufacturing Dive](https://www.manufacturingdive.com/news/3m-q1-2025-850-million-tariff-impact-trade-war-china/746037/) .

4. **New Product Launch**: 3M recently introduced a new abrasives catalog in June 2025, showcasing its latest innovations [NEW 3M Abrasives Catalog (June 2025) - Cline Tool](https://clinetool.com/blog/2025/06/23/new-3m-abrasives-catalog-june-2025/) .

These updates highlight 3M's focus on innovation, operational efficiency, and navigating global trade challenges.